In [2]:
from IPython.core.debugger import set_trace
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [3]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [10]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    n = 0
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                n+=1
                print("n",n)
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value. Ref: Sutton book eq. 4.6.
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
            print(delta)
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)

In [11]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

n 1
n 2
n 3
n 4
0
n 5
n 6
n 7
n 8
1.0
n 9
n 10
n 11
n 12
1.25
n 13
n 14
n 15
n 16
1.3125
n 17
n 18
n 19
n 20
1.3125
n 21
n 22
n 23
n 24
1.5
n 25
n 26
n 27
n 28
1.6875
n 29
n 30
n 31
n 32
1.75
n 33
n 34
n 35
n 36
1.75
n 37
n 38
n 39
n 40
1.75
n 41
n 42
n 43
n 44
1.84375
n 45
n 46
n 47
n 48
1.8984375
n 49
n 50
n 51
n 52
1.8984375
n 53
n 54
n 55
n 56
1.8984375
n 57
n 58
n 59
n 60
1.8984375
n 61
n 62
n 63
n 64
1.8984375
n 65
n 66
n 67
n 68
0
n 69
n 70
n 71
n 72
0.9375
n 73
n 74
n 75
n 76
1.296875
n 77
n 78
n 79
n 80
1.41796875
n 81
n 82
n 83
n 84
1.41796875
n 85
n 86
n 87
n 88
1.41796875
n 89
n 90
n 91
n 92
1.55078125
n 93
n 94
n 95
n 96
1.654296875
n 97
n 98
n 99
n 100
1.654296875
n 101
n 102
n 103
n 104
1.654296875
n 105
n 106
n 107
n 108
1.724609375
n 109
n 110
n 111
n 112
1.724609375
n 113
n 114
n 115
n 116
1.724609375
n 117
n 118
n 119
n 120
1.724609375
n 121
n 122
n 123
n 124
1.724609375
n 125
n 126
n 127
n 128
1.724609375
n 129
n 130
n 131
n 132
0
n 133
n 134
n 135
n 136
0.88671875


In [9]:
v

array([  0.        , -13.99993529, -19.99990698, -21.99989761,
       -13.99993529, -17.9999206 , -19.99991379, -19.99991477,
       -19.99990698, -19.99991379, -17.99992725, -13.99994569,
       -21.99989761, -19.99991477, -13.99994569,   0.        ])

In [5]:
print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Value Function:
[  0.         -13.99993529 -19.99990698 -21.99989761 -13.99993529
 -17.9999206  -19.99991379 -19.99991477 -19.99990698 -19.99991379
 -17.99992725 -13.99994569 -21.99989761 -19.99991477 -13.99994569
   0.        ]

Reshaped Grid Value Function:
[[  0.         -13.99993529 -19.99990698 -21.99989761]
 [-13.99993529 -17.9999206  -19.99991379 -19.99991477]
 [-19.99990698 -19.99991379 -17.99992725 -13.99994569]
 [-21.99989761 -19.99991477 -13.99994569   0.        ]]



In [6]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)